# import necessary library

In [8]:
# for garbage collection
import gc

# for warnings
import warnings
warnings.filterwarnings("ignore")

# utility libraries
import os
import copy
import tqdm
import numpy as np 
import pandas as pd 
import cv2, random, time, shutil, csv
import tensorflow as tf 
import math
import time
from util import *

# keras libraries
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

In [7]:
# checking if GPU is being used for training
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("GPU is not detected")

Default GPU Device:/device:GPU:0


# prepare of data set


In [9]:
# loading labels from csv file
data_dir = os.path.join(os.getcwd(), 'Datasets')
data_df = pd.read_csv(os.path.join(data_dir, 'labels.csv'))
breeds = sorted(data_df.breed.unique())

print(f"Shape\t\t: {data_df.shape}")
print(f"Unique labels\t: {len(breeds)}")

Shape		: (10222, 2)
Unique labels	: 120


In [21]:
# hyperparameters
img_size = 299
batch_size=128
epochs=80

In [22]:
# load images data
def loadData():
    images_list = sorted(os.listdir(os.path.join(data_dir, 'train')))
    X = []
    Y = []
    for image in tqdm.tqdm(images_list):
        # find matching label
        breed = data_df[data_df['id'] == image[:-4]].iloc[0,1]
        breed_index = int(breeds.index(breed)) 

        # reading image data
        image_path = os.path.join(data_dir, 'train', image)
        res_image = load_img(image_path, target_size=(img_size, img_size))
        res_image = np.asarray(res_image)
        X.append(res_image)
        Y.append(breed_index)
    return X, Y

In [23]:
saving_path = os.path.join(data_dir, f'data_resized{img_size}x{img_size}.h5')


t = time.time()
if (os.path.isfile(saving_path)):
    X, Y = load_h5_data(saving_path)
else:
    X, Y = loadData()
    generate_h5_data(X, Y, saving_path)
print(f'Time taken: {time.time()-t}')


100%|███████████████████████████████████████████████████████████████████████████| 10222/10222 [01:02<00:00, 164.21it/s]


Unable to allocate 2.55 GiB for an array with shape (10222, 299, 299, 3) and data type uint8
Time taken: 65.90514922142029


In [17]:
# Converting to arrays
print(len(X), len(Y))
assert(len(X)==len(Y))
Xarr = np.array(X)
Yarr = np.array(Y).reshape(-1,1)

del(X)
print(Xarr.shape, Yarr.shape)
gc.collect()

10222 10222
(10222, 224, 224, 3) (10222, 1)


143

In [19]:
# print out shape, dtype and data size
nparray_info("\nXarr", Xarr)
nparray_info("\nYarr", Yarr)


Xarr
-  shape	 (10222, 224, 224, 3)
-  dtype	 uint8
- nbytes	 1538697216 (1.43 GB)

Yarr
-  shape	 (10222, 1)
-  dtype	 int32
- nbytes	 40888 (39.93 KB)


In [20]:
# converting labels to one hot
Yarr_hot = to_categorical(Y)
print(Xarr.shape, Yarr_hot.shape)

(10222, 224, 224, 3) (10222, 120)


# Feature Extraction

In [ ]:
# FEATURE EXTRACTION OF TRAINING ARRAYS
AUTO = tf.data.experimental.AUTOTUNE
def get_features(model_name, data_preprocessor, data):
    '''
    1- Create a feature extractor to extract features from the data.
    2- Returns the extracted features and the feature extractor.
    '''
    dataset = tf.data.Dataset.from_tensor_slices(data)


    def preprocess(x):
        x = tf.image.random_flip_left_right(x)
        x = tf.image.random_brightness(x, 0.5)
        return x

    ds = dataset.map(preprocess, num_parallel_calls=AUTO).batch(64)

    input_size = data.shape[1:]
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)

    base_model = model_name(weights='imagenet', include_top=False,
                                input_shape=input_size)(preprocessor)

    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)


    #Extract feature.
    feature_maps = feature_extractor.predict(ds, verbose=1)
    nparray_info("Feature maps", feature_maps)
    
    # deleting variables
    del(feature_extractor, base_model, preprocessor, dataset)
    gc.collect()
    return feature_maps

In [ ]:
# FEATURE EXTRACTION OF VALIDAION AND TESTING ARRAYS
def get_valfeatures(model_name, data_preprocessor, data):
    '''
    Same as above except not image augmentations applied.
    Used for feature extraction of validation and testing.
    '''

    dataset = tf.data.Dataset.from_tensor_slices(data)

    ds = dataset.batch(64)

    input_size = data.shape[1:]
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)

    base_model = model_name(weights='imagenet', include_top=False, input_shape=input_size)(preprocessor)

    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    #Extract feature.
    feature_maps = feature_extractor.predict(ds, verbose=1)
    nparray_info("Feature maps", feature_maps)
    return feature_maps

In [ ]:
# generate feature dictionary 
def generate_model_features(feat_func, model_names, models, preprocs, array):
    print(f"Beggining extraction with {feat_func.__name__}\n")
    feats_dict = {}

    for i in range(len(models)):
        print(f"\nStarting feature extraction with {models[i].__name__} using {preprocs[i].__name__}\n")
        # applying the above function and storing in list
        feats_dict[model_names[i]] = feat_func(models[i], preprocs[i], array)
        
    # memory saving
    del(array)
    gc.collect()
    return feats_dict

In [ ]:
# RETURNING CONCATENATED FEATURES
def get_concat_features(model_names, feats_dict):
    feats_list = []
    
    for name in model_names:
        print(f"collect features for {name}...")
        feats_list.append( feats_dict[name] )

    # features concatenating
    final_feats = np.concatenate(feats_list, axis=-1)
    # memory saving
    del(feats_list)
    gc.collect()
    return final_feats

In [ ]:
# DEFINING models and preprocessors imports
from tensorflow.keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input
from tensorflow.keras.applications.densenet import DenseNet169, preprocess_input
densenet_preprocessor = preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input 
mobilenet_preprocessor = preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
restnet_preprocessor = preprocess_input


# collecting top 6 models' features for evaluation later
model_names = ["Xception", "DenseNet169", "InceptionResNetV2", "MobileNet", "InceptionV3", "RestNet50"]
models = [Xception, DenseNet169, InceptionResNetV2, MobileNet, InceptionV3, ResNet50 ]
preprocs = [xception_preprocessor, densenet_preprocessor, inc_resnet_preprocessor, 
            mobilenet_preprocessor, inception_preprocessor, restnet_preprocessor]

In [ ]:
# Perform feature extraction and save as h5 file
saving_path = os.path.join(data_dir, f'features.h5')

t = time.time()
# feats_dict = {}
if (os.path.isfile(saving_path)):
    feats_dict = load_features_data(saving_path)
else:
    feats_dict = generate_model_features(get_features, model_names, models, preprocs, Xarr)
    generate_features_data(feats_dict, saving_path)
gc.collect()
print(f'Time taken: {time.time()-t}')

In [ ]:
for name in feats_dict:
    nparray_info(name, feats_dict[name])

In [ ]:
# collect features of from dict
train_models = [model_names[i] for i in range(3)]    # can replace range to [0,1,2]
final_train_features = get_concat_features(train_models, feats_dict)

gc.collect()
nparray_info("Final feature maps", final_train_features)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
EarlyStop_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=0)

my_callback=[EarlyStop_callback]

In [ ]:
from sklearn.model_selection import StratifiedKFold
# Use K fold of 5, to ensure train:validation is 80:20
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=10).split(final_train_features, Y))

trained_models = []
val_accuracy = []
val_losses = []
train_accuracy = []
train_losses = []

#Prepare And Train DNN model
for i, (train_idx, valid_idx) in enumerate(splits): 

    print(f"\nStarting fold {i+1}\n")
    x_train_fold = final_train_features[train_idx, :]
    y_train_fold = Yarr_hot[train_idx, :]
    x_val_fold = final_train_features[valid_idx]
    y_val_fold = Yarr_hot[valid_idx, :]

    dnn = keras.models.Sequential([
        InputLayer(final_train_features.shape[1:]),
        Dropout(0.7),
        Dense(120, activation='softmax')
    ])

    dnn.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    print("Training...")
    #Train simple DNN on extracted features.
    h = dnn.fit(x_train_fold, y_train_fold,
                batch_size=batch_size,
                epochs=epochs,
                verbose=0,
                validation_data = (x_val_fold, y_val_fold),
                callbacks=my_callback)
    print("Evaluating model ...")
    model_res_val = dnn.evaluate(x_val_fold, y_val_fold)
    model_res_train = dnn.evaluate(x_train_fold, y_train_fold)
    train_accuracy.append(model_res_train[1])
    train_losses.append(model_res_train[0])
    val_accuracy.append(model_res_val[1])
    val_losses.append(model_res_val[0])
    trained_models.append(dnn)
    plot_model_history(h, saving_name=f"fold_{i+1}_LA")
    

print('\n CV Score -')
print(f"\nTrainAccuracy - avg:{sum(train_accuracy)/len(train_accuracy)}, max:{max(train_accuracy)}")
print(f"\nTrainLoss - avg:{sum(train_losses)/len(train_losses)}, min:{min(train_losses)}")
print(f"\nValAccuracy - avg:{sum(val_accuracy)/len(val_accuracy)}, max:{max(val_accuracy)}")
print(f"\nValLoss - avg:{sum(val_losses)/len(val_losses)}, min:{min(val_losses)}")


In [ ]:
# # TEST IMAGES
# test_images_list = sorted(os.listdir(os.path.join(data_dir, 'test')))
# X = []
# i = 0
# for image in tqdm.tqdm(test_images_list):

#     image_path = os.path.join(data_dir, 'test',image)
#     orig_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
#     res_image = cv2.resize(orig_image,(img_size, img_size))
#     X.append(res_image)
#     i+=1

# Xtesarr = np.array(X)

# del(X)
# gc.collect()

# Xtesarr.shape

In [ ]:
# # FEATURE EXTRACTION OF TEST IMAGES
# test_features = get_concat_features(get_valfeatures, models, preprocs, Xtesarr)

# del(Xtesarr)
# gc.collect()
# print('Final feature maps shape', test_features.shape)

In [ ]:
# y_pred_norm = trained_models[0].predict(test_features, batch_size=128)/3
# for dnn in trained_models[1:]:
#     y_pred_norm += dnn.predict(test_features, batch_size=128)/3

# y_pred_norm.shape

# df.iloc[:, 1:] = y_pred_norm
# df.to_csv('submission.csv')
